In [ ]:
# importo librerias necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import Pipeline
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model

# xgboost regressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
#se importa el set de datos
url = 'https://raw.githubusercontent.com/Jorge-89/Regresion-Lineal-/main/base_datos_estaciones_met_V3_train.csv'
df = pd.read_csv(url, sep=",")
df.columns

Index(['Unnamed: 0', 'fecha_completa', 'Fecha', 'Hora',
       'Temp_Alicia AgriculturaCba', 'Humedad_Alicia AgriculturaCba',
       'PP_Alicia AgriculturaCba', 'Temp_Las Varas AgriculturaCba',
       'Humedad_Las Varas AgriculturaCba', 'PP_Las Varas AgriculturaCba',
       'Temp_San Miguel - Establecimiento Don Luis',
       'Humedad_San Miguel - Establecimiento Don Luis',
       'PP_San Miguel - Establecimiento Don Luis',
       'Temp_San Miguel - Listello', 'Humedad_San Miguel - Listello',
       'PP_San Miguel - Listello', 'Temp_ San Miguel - Las Varillas',
       'Humedad_ San Miguel - Las Varillas', 'PP_ San Miguel - Las Varillas'],
      dtype='object')

In [ ]:
columnas = ['Temp_Las Varas AgriculturaCba','Humedad_Las Varas AgriculturaCba',
            'Temp_San Miguel - Establecimiento Don Luis','Humedad_San Miguel - Establecimiento Don Luis',
            'Temp_San Miguel - Listello','Humedad_San Miguel - Listello','Temp_ San Miguel - Las Varillas','Humedad_ San Miguel - Las Varillas']

x = df[columnas]

y = df['Humedad_Alicia AgriculturaCba']

In [ ]:
# Separo los datos de "train" en entrenamiento y prueba "test" para probar los algoritmos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
def get_scores_test(model):
  """
  Función que obtiene el MSE en test
  """
  mse = mean_squared_error(
          y_true=y_test,
          y_pred = model.predict(x_test),
          squared = True
          )
  
  print(f'El error (MSE) en test es: {mse}')
    

In [ ]:
def get_cv_scores_train(model):
  """
  Función que obtiene el MSE en train
  """
  scores = cross_val_score(
                  model,
                  X = x_train,
                  y = y_train,
                  cv = 5,
                  scoring = 'neg_mean_squared_error'
                  )
  print('El error (MSE) en train es: ',scores.mean()*(-1))


## Linear Regression

In [ ]:
modelo = Pipeline([('scaler', StandardScaler()),('features', PolynomialFeatures()),
                    ('regression', LinearRegression(fit_intercept=True))
                    ])
hyperparameters={'features__degree':[1,2,3,4,5,6]}
grilla=GridSearchCV(modelo,hyperparameters,refit=True, cv=5,scoring='neg_mean_squared_error')
grilla.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('features', PolynomialFeatures()),
                                       ('regression', LinearRegression())]),
             param_grid={'features__degree': [1, 2, 3, 4, 5, 6]},
             scoring='neg_mean_squared_error')

In [ ]:
grilla.best_params_

{'features__degree': 3}

In [ ]:
grilla.best_score_

-9.9807242604592

In [ ]:
grilla.cv_results_['mean_test_score']

array([-1.27932382e+01, -1.07428487e+01, -9.98072426e+00, -3.22124400e+01,
       -6.77914356e+02, -5.29716949e+05])

In [ ]:
# Fit best model
modelo=grilla.best_estimator_
modelo.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('features', PolynomialFeatures(degree=3)),
                ('regression', LinearRegression())])

In [ ]:
# Evalúo train
get_cv_scores_train(modelo)

# Evalúo test
get_scores_test(modelo)

El error (MSE) en train es:  9.9807242604592
El error (MSE) en test es: 3.212395028467499


Código prueba con squared = True

In [ ]:
modelo = Pipeline([('scaler', StandardScaler()),('features', PolynomialFeatures(degree=3)),
                    ('regression', LinearRegression(fit_intercept=True))
                    ])
modelo.fit(x_train, y_train)
# Evalúo train
get_cv_scores_train(modelo)

# Evalúo test
get_scores_test(modelo)

El error (MSE) en train es:  9.9807242604592
El error (MSE) en test es: 10.319481818922702


El mejor modelo es de regresión lineal simple de grado 3

## Pruebo quitando valores de temperatura

In [ ]:
columnas = ['Humedad_Las Varas AgriculturaCba',
            'Humedad_San Miguel - Establecimiento Don Luis',
            'Humedad_San Miguel - Listello','Humedad_ San Miguel - Las Varillas']

x = df[columnas]

y = df['Humedad_Alicia AgriculturaCba']

In [ ]:
# Separo los datos de "train" en entrenamiento y prueba "test" para probar los algoritmos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
modelo = Pipeline([('scaler', StandardScaler()),('features', PolynomialFeatures()),
                    ('regression', LinearRegression(fit_intercept=True,normalize=True))
                    ])
hyperparameters={'features__degree':[1,2,3,4,5,6]}
grilla=GridSearchCV(modelo,hyperparameters,refit=True, cv=5,scoring='neg_mean_squared_error')
grilla.fit(x_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('features',
                                        PolynomialFeatures(degree=2,
                                                           include_bias=True,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('regression',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                                   

In [ ]:
grilla.best_params_

{'features__degree': 5}

In [ ]:
# Fit best model
modelo=grilla.best_estimator_
modelo.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('features',
                 PolynomialFeatures(degree=5, include_bias=True,
                                    interaction_only=False, order='C')),
                ('regression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=True))],
         verbose=False)

In [ ]:
# Evalúo en el conjunto de testeo

predicciones = modelo.predict(X = x_test)
#print(predicciones[0:3,])
rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {np.sqrt(rmse)}")

El error (rmse) de test es: 1.7987044530565228


In [ ]:
# Evalúo con cross validation
get_cv_scores(modelo)

CV Mean:  10.83244986854046
STD:  0.6149557134901172




Conclusión: el error es más chico cuando se consideran los valores de temperatura

# Regression con Ridge

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
modelo2 = Pipeline([('scaler', StandardScaler()),
                    ('features', PolynomialFeatures()),
                  ('model',Ridge())
])

grilla = GridSearchCV(modelo2,
                      {'model__alpha':[0,0.001,0.0001,0.00001], 'features__degree':[1,2,3,4,5,6]}, refit=True,
                      cv = 5, scoring="neg_mean_squared_error", verbose=3
                      )
grilla.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END features__degree=1, model__alpha=0;, score=-12.766 total time=   0.0s
[CV 2/5] END features__degree=1, model__alpha=0;, score=-13.334 total time=   0.0s
[CV 3/5] END features__degree=1, model__alpha=0;, score=-11.952 total time=   0.0s
[CV 4/5] END features__degree=1, model__alpha=0;, score=-12.273 total time=   0.0s
[CV 5/5] END features__degree=1, model__alpha=0;, score=-13.641 total time=   0.0s
[CV 1/5] END features__degree=1, model__alpha=0.001;, score=-12.766 total time=   0.0s
[CV 2/5] END features__degree=1, model__alpha=0.001;, score=-13.334 total time=   0.0s
[CV 3/5] END features__degree=1, model__alpha=0.001;, score=-11.952 total time=   0.0s
[CV 4/5] END features__degree=1, model__alpha=0.001;, score=-12.273 total time=   0.0s
[CV 5/5] END features__degree=1, model__alpha=0.001;, score=-13.641 total time=   0.0s
[CV 1/5] END features__degree=1, model__alpha=0.0001;, score=-12.766 total time=   0.0s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.10988e-16): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


[CV 1/5] END features__degree=6, model__alpha=1e-05;, score=-4054.382 total time=   2.5s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.08605e-16): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


[CV 2/5] END features__degree=6, model__alpha=1e-05;, score=-2243.368 total time=   2.5s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.0314e-16): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


[CV 3/5] END features__degree=6, model__alpha=1e-05;, score=-228.034 total time=   2.5s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.01542e-16): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


[CV 4/5] END features__degree=6, model__alpha=1e-05;, score=-163.781 total time=   2.5s
[CV 5/5] END features__degree=6, model__alpha=1e-05;, score=-2164.965 total time=   2.5s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('features', PolynomialFeatures()),
                                       ('model', Ridge())]),
             param_grid={'features__degree': [1, 2, 3, 4, 5, 6],
                         'model__alpha': [0, 0.001, 0.0001, 1e-05]},
             scoring='neg_mean_squared_error', verbose=3)

In [ ]:
grilla.best_params_

{'features__degree': 3, 'model__alpha': 0.001}

In [ ]:
grilla.best_score_

-9.946341701038815

In [ ]:
# Fit best model
modelo3=grilla.best_estimator_
modelo3.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('features', PolynomialFeatures(degree=3)),
                ('model', Ridge(alpha=0.001))])

In [ ]:
# Evalúo train
get_cv_scores_train(modelo3)

# Evalúo test
get_scores_test(modelo3)

El error (MSE) en train es:  9.946341701038815
El error (MSE) en test es: 3.2032196353279065


Código prueba con squared = True

In [ ]:
modelo = Pipeline([('scaler', StandardScaler()),('features', PolynomialFeatures(degree=3)),
                    ('model', Ridge(alpha=0.001,fit_intercept=True))
                    ])
modelo.fit(x_train, y_train)
# Evalúo train
get_cv_scores_train(modelo)

# Evalúo test
get_scores_test(modelo)

El error (MSE) en train es:  9.946341701038815
El error (MSE) en test es: 10.260616032150248


# Regresión con Lasso

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
modelo= Pipeline([ ('scaler', StandardScaler()),
                    ('features', PolynomialFeatures()),
                     ('model',Lasso())
])

#Ya probe con varios alpha y grados de polinomio, fui dejando los que mas cerca estan de valores menores de MSE
grilla = GridSearchCV(modelo,
                      {'model__alpha':[0.001,0.0001,0.00001,0.000001], 'features__degree':[1,2,3,4,5,6]},
                      refit=True,cv = 5, scoring="neg_mean_squared_error")
grilla.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.028e+02, tolerance: 3.130e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.496e+02, tolerance: 3.129e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('features', PolynomialFeatures()),
                                       ('model', Lasso())]),
             param_grid={'features__degree': [1, 2, 3, 4, 5, 6],
                         'model__alpha': [0.001, 0.0001, 1e-05, 1e-06]},
             scoring='neg_mean_squared_error')

In [ ]:
 print(grilla.best_params_)
print(grilla.best_score_)

{'features__degree': 6, 'model__alpha': 1e-06}
-10.161335958376018


In [ ]:
# Fit best model
modelo2=grilla.best_estimator_
modelo2.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.051e+04, tolerance: 3.906e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Pipeline(steps=[('scaler', StandardScaler()),
                ('features', PolynomialFeatures(degree=6)),
                ('model', Lasso(alpha=1e-06))])

In [ ]:
# Evalúo train
get_cv_scores_train(modelo2)

# Evalúo test
get_scores_test(modelo2)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.233e+04, tolerance: 3.130e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.178e+04, tolerance: 3.111e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

El error (MSE) en train es:  10.161335958376018
El error (MSE) en test es: 3.2409801425113516


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.190e+04, tolerance: 3.098e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Código prueba última

In [ ]:
modelo = Pipeline([('scaler', StandardScaler()),('features', PolynomialFeatures(degree=6)),
                    ('model', Lasso(alpha=0.000001,fit_intercept=True))
                    ])
modelo.fit(x_train, y_train)
# Evalúo train
get_cv_scores_train(modelo)

# Evalúo test
get_scores_test(modelo)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.051e+04, tolerance: 3.906e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.233e+04, tolerance: 3.130e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

El error (MSE) en train es:  10.161335958376018
El error (MSE) en test es: 10.5039522841529


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.190e+04, tolerance: 3.098e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


El mejor modelo de regresión lineal a utilizar es con la penalización Ridge